In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import numpy as np
import pandas as pd
import pprint
import math
import datetime
pd.options.display.max_columns = 999

username = 'a8e0caf507870e4d91e140d75cb1e0c5'
password = '38b5c1a103fc779e956eab1436dc1934'

url_products = 'https://paulas-choice-malaysia.myshopify.com/admin/products.json?limit=250'
products_request = requests.get(url_products, auth=HTTPBasicAuth(username, password)).text
products_json = json.loads(products_request)
products_df = pd.io.json.json_normalize(products_json, record_path='products').drop(['options', 'admin_graphql_api_id', 'body_html', 'image', 'images', 'vendor', 'published_scope', 'template_suffix', 'created_at', 'updated_at', 'published_at'], axis = 1)
products_df = products_df[products_df.handle.str.contains('pcm-rewards') == False]
# products_df.shape
# products_df['variants'][0][0]['inventory_item_id']

products_df_concat = pd.concat([products_df.drop(['id', 'variants'], axis = 1), products_df.variants.apply(pd.Series)], axis =1)
products_df_melt = products_df_concat.melt(id_vars=products_df_concat.columns[:-8])#.rename(columns = {'id': 'id_1', 'title': 'title_1'} )#.dropna(subset = ['created_at'])

# Expand value column (dict entries) into multiple columns using pd.Series apply function
# products_df_melt.value.apply(pd.Series)
products_df_melt = pd.concat([products_df_melt, products_df_melt.value.apply(pd.Series)], axis =1 , join = 'outer').drop(['value', 'admin_graphql_api_id', 'options'], axis = 1)# .drop(['index', 'value', 'id_1'], axis =1 )

products_df_melt = products_df_melt.dropna(subset = ['created_at'])

# To convert numbers stored in scientific notation into legible text strings
def try_float(x):
    try:
        return str(format(float(x), '.0f'))
    except ValueError as e:
        return x

# Convert columns in scientific notation to text
products_df_melt['id'] = products_df_melt['id'].apply(try_float).astype('str')
products_df_melt['image_id'] = products_df_melt['image_id'].apply(try_float).astype('str')
products_df_melt['product_id'] = products_df_melt['product_id'].apply(try_float).astype('str')
products_df_melt['inventory_item_id'] = products_df_melt['inventory_item_id'].apply(try_float).astype('str')

# Convert columns in text to numbers
products_df_melt['price'] = products_df_melt['price'].astype(float)

# Convert date-time columns to datetime format
products_df_melt['created_at'] = products_df_melt['created_at'].astype("datetime64[ns]")
# products_df_melt['published_at'] = products_df_melt['published_at'].astype("datetime64[ns]")
products_df_melt['updated_at'] = products_df_melt['updated_at'].astype("datetime64[ns]")

products_df_melt['snapshot_datetime'] = datetime.datetime.now()
products_df_melt

C:\Program Files\Python36\lib\site-packages\pandas\core\indexes\api.py:57: RuntimeWarning: '>' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
C:\Program Files\Python36\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '>' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


,handle,product_type,tags,title,variable,barcode,compare_at_price,created_at,fulfillment_service,grams,id,image_id,inventory_item_id,inventory_management,inventory_policy,inventory_quantity,old_inventory_quantity,option1,option2,option3,position,price,product_id,requires_shipping,sku,taxable,title,updated_at,weight,weight_unit,0,snapshot_datetime
0,age-rewind-retinol-kit,,,Age-Rewind Retinol Kit,0,,525.00,2018-06-11 13:56:32,manual,300.0,7891418120240,nan,8097681440816,shopify,deny,10.0,10.0,Default Title,NaN,NaN,1.0,439.0,826061357104,True,23ANNIV08,True,Default Title,2018-07-11 13:22:30,0.30,kg,NaN,2018-07-18 13:57:08.631216
1,all-over-hair-body-shampoo,Hair & Body,"All Skin Types, Basic, Body, Cleanser, Dehydra...",All Over Hair & Body Shampoo,0,655439050005,None,2018-01-24 01:51:24,manual,487.0,1356347867184,713336487984,1384876572720,shopify,deny,6.0,6.0,429 ml,NaN,NaN,1.0,95.0,162814558256,True,5000,True,429 ml,2018-06-17 08:20:20,487.00,g,NaN,2018-07-18 13:57:08.631216
2,anti-aging-trio,Kit,,Anti Aging Trio,0,,None,2018-01-24 01:45:57,manual,0.0,1356303532080,nan,1384832237616,shopify,deny,1.0,1.0,Default,NaN,NaN,1.0,149.0,162805776432,True,XMAS2,True,Default,2018-04-27 07:37:23,0.00,g,NaN,2018-07-18 13:57:08.631216
3,barrier-repairing-omega-3-6-9-duo,,,Barrier-Repairing Omega 3-6-9 Duo,0,,375.00,2018-06-11 13:14:40,manual,200.0,7891390464048,nan,8097593688112,shopify,deny,17.0,17.0,Default Title,NaN,NaN,1.0,319.0,826055557168,True,23ANNIV03,True,Default Title,2018-07-11 19:15:20,0.20,kg,NaN,2018-07-18 13:57:08.631216
4,body-renewal-special,Kit,,Body Renewal,0,,None,2018-01-24 01:45:55,manual,0.0,1356303433776,nan,1384832139312,shopify,deny,-177.0,-177.0,Default,NaN,NaN,1.0,78.0,162805710896,True,XMAS1,True,Default,2018-04-27 07:37:23,0.00,g,NaN,2018-07-18 13:57:08.631216
5,brightening-trio,Kit,,Brightening Trio,0,,None,2018-01-24 01:45:50,manual,0.0,1356303237168,nan,1384831942704,shopify,deny,8.0,8.0,Default,NaN,NaN,1.0,98.0,162805579824,True,XMAS4,True,Default,2018-04-27 07:37:24,0.00,g,NaN,2018-07-18 13:57:08.631216
6,calmness-relief-serum,Treatment,"Acne, All Skin Types, Basic, Dullness, Face, G...",CALM Repairing Sensitive Serum,0,655439037006,None,2018-01-24 01:48:32,manual,66.0,1356318015536,713321185328,1384846786608,shopify,deny,10.0,10.0,30 ml,NaN,NaN,1.0,165.0,162809282608,True,3700,True,30 ml,2018-07-16 02:52:00,66.00,g,NaN,2018-07-18 13:57:08.631216
7,calm-redness-relief-kit-normal-to-dry,Kit,"Bundle, Dry to Very Dry, Sensitive",CALM Sensitive (Normal to Dry) Advanced Kit,0,,830.00,2018-01-24 01:47:48,manual,850.0,1356314116144,nan,1384842821680,shopify,deny,3.0,3.0,Default,NaN,NaN,1.0,749.0,162808135728,True,4170,True,Default,2018-05-24 05:03:05,850.00,g,NaN,2018-07-18 13:57:08.631216
8,calm-redness-relief-kit-normal-to-oily,Kit,"Bundle, Combination, Oily, Sensitive",CALM Sensitive (Normal to Oily) Advanced Kit,0,,830.00,2018-01-24 01:47:49,manual,850.0,1356314148912,nan,1384842854448,shopify,deny,3.0,3.0,Default,NaN,NaN,1.0,749.0,162808168496,True,4180,True,Default,2018-06-03 14:33:15,850.00,g,NaN,2018-07-18 13:57:08.631216
9,calm-redness-relief-1-bha-lotion,Exfoliator,"Basic, Clogged Pores, Dry to Very Dry, Exfolia...",CALM Sensitive 1% BHA Lotion,0,655439091008,None,2018-01-24 01:48:28,manual,140.0,1356317884464,713320923184,1384846590000,shopify,deny,15.0,15.0,100 ml,NaN,NaN,1.0,130.0,162809184304,True,9100,True,100 ml,2018-07-13 08:07:05,140.00,g,NaN,2018-07-18 13:57:08.631216


In [56]:
pcsg_shopify = pd.read_csv('orders_export - Copy.csv', low_memory=False, 
                           dtype={'Lineitem sku': 'str', 'Name': 'str'},
                           parse_dates = ['Paid at', 'Created at'])

# To simplify columns
# new_columns = ['Name', 'Email', 'Financial Status', 'Paid at', 'Fulfillment Status',
#       'Fulfilled at', 'Accepts Marketing', 'Currency', 'Subtotal', 'Shipping',
#       'Taxes', 'Total', 'Discount Code', 'Shipping Method',
#       'Created at', 'Lineitem quantity', 'Lineitem name', 'Lineitem price',
#      'Lineitem compare at price', 'Lineitem sku',
#       'Lineitem requires shipping', 'Lineitem taxable',
#       'Lineitem fulfillment status', 'Shipping City', 'Shipping Zip', 'Shipping Province',
#       'Shipping Country', 'Shipping Phone',
#       'Cancelled at', 'Payment Method', 'Payment Reference',
#       'Refunded Amount', 'Vendor']
# pcsg_shopify = pcsg_shopify[new_columns]

# Import master list
sku_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx", sheet_name='SKU List' )

# Import bundle sheet of masterlist file
bundle_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx",
                              sheet_name='Bundles').rename(columns = {'Parent SKU': "parent_sku",
                                                                      'Parent Name': 'parent_name',
                                                                      'Quantity': 'child_quantity',
                                                                      'Child SKU': "child_sku",
                                                                      'Child Name': 'child_name'}).astype({'child_sku': 'str',
                                                                                                           'parent_sku': 'str'})
# Use (how = "Left") in order to duplicate the line item information on each composite SKU
pcsg_shopify_merge = pd.merge(pcsg_shopify, bundle_master, left_on="Lineitem sku", right_on = 'parent_sku', how='left')

# fill those NAs in parent_sku column (i.e. individual SKUs)  with their original sku names
pcsg_shopify_merge['parent_sku'].fillna(pcsg_shopify_merge['Lineitem sku'], inplace=True)
pcsg_shopify_merge['child_sku'].fillna(pcsg_shopify_merge['Lineitem sku'], inplace=True)


# pcsg_shopify_merge['total_quantity'].fillna(pcsg_shopify_merge['Lineitem quantity'], inplace = True)
# Calculates the actual quantity of composite SKUs sold (since, e.g., there might be more than one 2010s sold in a bundle)
pcsg_shopify_merge['child_subtotal_quantity'] =  pcsg_shopify_merge['child_quantity'] * pcsg_shopify_merge['Lineitem quantity']
# same concept, repeat the quantities with individual SKUs in the child_quantity column
pcsg_shopify_merge['child_subtotal_quantity'].fillna(pcsg_shopify_merge['Lineitem quantity'], inplace=True)



In [65]:
pcsg_shopify_merge['bundle_status'] = pcsg_shopify_merge['parent_name'].apply(lambda x: 0 if pd.isna(x) else 1)

In [66]:
pcsg_shopify_merge

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,parent_sku,parent_name,child_sku,child_name,child_quantity,Unit Price,Price,Component Retail,Bundle Retail,% Discount,Discount Unit Price,child_subtotal_quantity,bundle_status
0,#PCMY2543,alicia_qing1110@yahoo.com.my,authorized,NaT,unfulfilled,NaN,yes,MYR,238.00,0.0,0.00,238.00,PCMY20,20.00,Pos Laju (Free Shipping on Orders above RM 150),2018-07-18 05:35:24,1,SKIN PERFECTING 2% BHA (Salicylic Acid) Liquid...,48.0,NaN,2016,True,True,pending,Alicia Ong Chuu Chyng,"C4-2-4, PSRN SG LONG 1, BDR SUNGAI LONG, Cypre...","C4-2-4, PSRN SG LONG 1, BDR SUNGAI LONG",Cypress condo,NaN,Kajang,'43000,SGR,MY,016-885 8320,Alicia Ong Chuu Chyng,"C4-2-4, PSRN SG LONG 1, BDR SUNGAI LONG, Cypre...","C4-2-4, PSRN SG LONG 1, BDR SUNGAI LONG",Cypress condo,NaN,Kajang,'43000,SGR,MY,016-885 8320,NaN,NaN,NaN,PayPal Express Checkout,c2716694708272.1,0.0,Paula's Choice,5.269520e+11,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,NaN,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
1,#PCMY2543,alicia_qing1110@yahoo.com.my,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-18 05:35:24,1,RESIST Daily Pore-Refining Treatment with 2% B...,60.0,NaN,7827,True,True,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7827,NaN,7827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
2,#PCMY2543,alicia_qing1110@yahoo.com.my,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-18 05:35:24,1,RESIST Weightless Body Treatment with 2% BHA (...,150.0,NaN,5700,True,True,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5700,NaN,5700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
3,#PCMY2542,liliannk@gmail.com,paid,2018-07-18 04:07:30,unfulfilled,NaN,yes,MYR,590.00,0.0,0.00,590.00,PCMY100,100.00,Pos Laju (Free Shipping on Orders above RM 150),2018-07-18 04:07:29,1,RESIST Daily Pore-Refining Treatment with 2% B...,160.0,NaN,7820,True,True,pending,Lilian Kwan,"Sri Murni, 8C-3A-5, 8 Lorong Kota Empat, Bukit...","Sri Murni, 8C-3A-5, 8 Lorong Kota Empat, Bukit...",NaN,NaN,Kuala Lumpur,'50480,SGR,MY,'+60162555807,Lilian Kwan,"Sri Murni, 8C-3A-5, 8 Lorong Kota Empat, Bukit...","Sri Murni, 8C-3A-5, 8 Lorong Kota Empat, Bukit...",NaN,NaN,Kuala Lumpur,'50480,SGR,MY,'+60162555807,NaN,NaN,NaN,Billplz,c2716580315184.2,0.0,Paula's Choice,5.269300e+11,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7820,NaN,7820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
4,#PCMY2542,liliannk@gmail.com,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-18 04:07:29,1,RESIST Intensive Wrinkle-Repair Retinol Serum ...,195.0,NaN,7710,True,True,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7710,NaN,7710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
5,#PCMY2542,liliannk@gmail.com,NaN,NaT,NaN,NaN

In [47]:
#pcsg_shopify_merge[pcsg_shopify_merge['parent_sku'] == '2016-4'] 
# pcsg_shopify_merge.columns

reorder_list = ['Name', 'Email', 'Financial Status', 'Paid at', 'Fulfillment Status', 'Subtotal', 'Shipping',
       'Taxes', 'Total', 'Discount Code', 'Shipping Method', 'Created at',
       'Lineitem compare at price', 'Lineitem requires shipping', 'Lineitem taxable',
      'Lineitem sku','Lineitem quantity', 'Lineitem name', 'Lineitem price', 'parent_sku', 'parent_name', 'child_sku',
       'child_name', 'child_quantity','child_subtotal_quantity' ,'Unit Price', 'Price',
       'Component Retail', 'Bundle Retail', '% Discount',
       'Discount Unit Price']

pcmy_simple_merge = pcsg_shopify_merge[reorder_list]
pcmy_simple_merge

,Name,Email,Financial Status,Paid at,Fulfillment Status,Subtotal,Shipping,Taxes,Total,Discount Code,Shipping Method,Created at,Lineitem compare at price,Lineitem requires shipping,Lineitem taxable,Lineitem sku,Lineitem quantity,Lineitem name,Lineitem price,parent_sku,parent_name,child_sku,child_name,child_quantity,child_subtotal_quantity,Unit Price,Price,Component Retail,Bundle Retail,% Discount,Discount Unit Price
0,#PCMY2543,alicia_qing1110@yahoo.com.my,authorized,NaT,unfulfilled,238.00,0.0,0.00,238.00,PCMY20,Pos Laju (Free Shipping on Orders above RM 150),2018-07-18 05:35:24,NaN,True,True,2016,1,SKIN PERFECTING 2% BHA (Salicylic Acid) Liquid...,48.0,2016,NaN,2016,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,#PCMY2543,alicia_qing1110@yahoo.com.my,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-18 05:35:24,NaN,True,True,7827,1,RESIST Daily Pore-Refining Treatment with 2% B...,60.0,7827,NaN,7827,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,#PCMY2543,alicia_qing1110@yahoo.com.my,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-18 05:35:24,NaN,True,True,5700,1,RESIST Weightless Body Treatment with 2% BHA (...,150.0,5700,NaN,5700,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,#PCMY2542,liliannk@gmail.com,paid,2018-07-18 04:07:30,unfulfilled,590.00,0.0,0.00,590.00,PCMY100,Pos Laju (Free Shipping on Orders above RM 150),2018-07-18 04:07:29,NaN,True,True,7820,1,RESIST Daily Pore-Refining Treatment with 2% B...,160.0,7820,NaN,7820,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,#PCMY2542,liliannk@gmail.com,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-18 04:07:29,NaN,True,True,7710,1,RESIST Intensive Wrinkle-Repair Retinol Serum ...,195.0,7710,NaN,7710,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,#PCMY2542,liliannk@gmail.com,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-18 04:07:29,NaN,True,True,7970,1,RESIST Skin Restoring Moisturizer with SPF 50 ...,155.0,7970,NaN,7970,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,#PCMY2542,liliannk@gmail.com,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-18 04:07:29,NaN,True,True,7640,1,RESIST Super Antioxidant Concentrate Serum wit...,180.0,7640,NaN,7640,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,#PCMY2541,bossbeautynail3851@gmail.com,paid,2018-07-18 02:50:10,unfulfilled,115.20,12.0,0.00,127.20,NaN,WholeSale Shipping to EM,2018-07-18 02:50:10,NaN,True,True,2140,1,RESIST Anti-Aging Eye Gel - 15ml,160.0,2140,NaN,2140,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
8,#PCMY2540,tenniech@gmail.com,paid,2018-07-18 01:44:33,unfulfilled,519.00,0.0,0.00,519.00,PCMY50,Pos Laju (Free Shipping on Orders above RM 150),2018-07-18 01:44:32,NaN,True,True,7970,1,RESIST Skin Restoring Moisturizer with SPF 50 ...,155.0,7970,NaN,7970,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,#PCMY2540,tenniech@gmail.com,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-18 01:44:32,NaN,True,True,7670,1,RESIST Advanced Replenishing Toner - 118 ml,125.0,7670,NaN,7670,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
pcsg_shopify_merge.to_csv("order_export_formatted_2.csv")

In [48]:
right  = pd.DataFrame(pcmy_simple_merge.groupby('child_sku')['child_subtotal_quantity'].sum()).reset_index()
left = pd.DataFrame(pcmy_simple_merge.groupby('Lineitem sku')['Lineitem quantity'].sum()).reset_index()
merge = pd.merge(left, right, how = "left", left_on = 'Lineitem sku', right_on =  'child_sku')
merge['difference'] = merge['Lineitem quantity'] - merge['child_subtotal_quantity']
merge[merge['difference'] != 0].sort_values('difference')

,Lineitem sku,Lineitem quantity,child_sku,child_subtotal_quantity,difference
30,2016,2,2016,110.0,-108.0
62,2659,16,2659,71.0,-55.0
70,2809,12,2809,52.0,-40.0
29,2010,162,2010,202.0,-40.0
127,6006,16,6006,49.0,-33.0
59,2609,6,2609,36.0,-30.0
182,7820,67,7820,94.0,-27.0
126,6002,47,6002,73.0,-26.0
136,6207,5,6207,25.0,-20.0
65,2759,3,2759,23.0,-20.0


In [51]:
pcmy_simple_merge[pcmy_simple_merge['child_sku'] == '2016'].to_excel('test.xlsx')